In [527]:
import pandas as pd
import pickle

In [528]:
file_path = '../data/cities.pkl'
with open(file_path, 'rb') as file:
    cities = pickle.load(file)


## Попробуем найти некоторые аномалии в данных

### Загрзим таблицу Sirena_export и приведем ее в удобный вид

In [531]:
sirena = pd.read_csv('../data/Sirena-export-fixed.csv', index_col='id')

sirena.columns = [
    'pax_name',
    'birth_date',
    'depart_date',
    'depart_time',
    'arrival_date',
    'arrival_time',
    'flight_number',
    'from',
    'dest',
    'ticket_number',
    'travel_doc',
    'seat',
    'meal',
    'class',
    'baggage',
    'pax_additional_info',
    'loyality',
    'agent_info'
]

sirena.head()

,pax_name,birth_date,depart_date,depart_time,arrival_date,arrival_time,flight_number,from,dest,ticket_number,travel_doc,seat,meal,class,baggage,pax_additional_info,loyality,agent_info
id,,,,,,,,,,,,,,,,,,
0,ОЗЕРОВ ИЛЬДАР ДАНИИЛОВИЧ,1999-05-15,2017-05-30,00:05,2017-05-30,08:05,SU1306NO,SVO,OVB,ZBQSPY7360415302044672,9375 053270,NaN,NaN,J,JGRPGN0PC,S,FF#SU 38116280,Go2See
1,КОЛОСОВ САМИР ТАМЕРЛАНОВИЧ,NaN,2017-12-27,02:15,2017-12-27,04:40,SU1323NO,MMK,SVO,MHPBBX7398421117936516,2244 645520,NaN,KSML,Y,YRSTUQ,9,FF#FB 284903754,NaN
2,ИГНАТОВА СНЕЖАНА КОНСТАНТИНОВНА,NaN,2017-09-19,06:40,2017-09-19,07:45,SU1481NO,KJA,SVO,REDLVB5174973140468001,8115 961316,NaN,NaN,Y,YSTNJL,F,NaN,KupiBilet
3,ЖАРОВ ПЛАТОН АЛЬБЕРТОВИЧ,1999-05-02,2017-03-18,22:10,2017-03-19,01:05,SU1180NO,SVO,VOG,NSJNGQ5274206497242737,98 6865148,NaN,NaN,J,JFLXLS0PC,#,FF#FB 884556993,Travelgenio
4,НИКОЛЬСКИЙ НИКОЛАЙ ИГОРЕВИЧ,1990-12-26,2017-03-18,22:10,2017-03-19,01:05,SU1180NO,SVO,VOG,VWNYGF6247422701565929,4396 926588,NaN,NaN,Y,YFLXPG,NaN,FF#SU 183142068,OZON.travel


### Оставим только не пустые и нужные колонки

In [533]:
sirena.count()

pax_name               155757
birth_date              77703
depart_date            155757
depart_time            155757
arrival_date           155757
arrival_time           155757
flight_number          155757
from                   155757
dest                   155757
ticket_number          155757
travel_doc             155757
seat                        0
meal                    58148
class                  155757
baggage                155757
pax_additional_info    124351
loyality                51917
agent_info             143814
dtype: int64

In [534]:
sirena_filtered = sirena[['flight_number', 'ticket_number', 'travel_doc']]

In [535]:
sirena_filtered.head()

,flight_number,ticket_number,travel_doc
id,,,
0,SU1306NO,ZBQSPY7360415302044672,9375 053270
1,SU1323NO,MHPBBX7398421117936516,2244 645520
2,SU1481NO,REDLVB5174973140468001,8115 961316
3,SU1180NO,NSJNGQ5274206497242737,98 6865148
4,SU1180NO,VWNYGF6247422701565929,4396 926588


### Добавим список городов в соответствии с кодами аэропортов

In [537]:
sirena_filtered['from'] = sirena['from'].map(cities)
sirena_filtered['dest'] = sirena['dest'].map(cities)

C:\Users\Hp\AppData\Local\Temp\ipykernel_5096\1082639287.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sirena_filtered['from'] = sirena['from'].map(cities)
C:\Users\Hp\AppData\Local\Temp\ipykernel_5096\1082639287.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sirena_filtered['dest'] = sirena['dest'].map(cities)


In [538]:
sirena_filtered.head()

,flight_number,ticket_number,travel_doc,from,dest
id,,,,,
0,SU1306NO,ZBQSPY7360415302044672,9375 053270,Moscow,Novosibirsk
1,SU1323NO,MHPBBX7398421117936516,2244 645520,Murmansk,Moscow
2,SU1481NO,REDLVB5174973140468001,8115 961316,Krasnoyarsk,Moscow
3,SU1180NO,NSJNGQ5274206497242737,98 6865148,Moscow,Volgograd
4,SU1180NO,VWNYGF6247422701565929,4396 926588,Moscow,Volgograd


### Преобразуем номер полета и номер билета для единого формата

In [540]:
sirena_filtered['flight_number']   = [str(number)[0:6].strip()    for number in sirena_filtered['flight_number'].values]
sirena_filtered['ticket_number']   = [str(ticket)[6:25].strip() for ticket in sirena_filtered['ticket_number'].values]

C:\Users\Hp\AppData\Local\Temp\ipykernel_5096\4017028316.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sirena_filtered['flight_number']   = [str(number)[0:6].strip()    for number in sirena_filtered['flight_number'].values]
C:\Users\Hp\AppData\Local\Temp\ipykernel_5096\4017028316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sirena_filtered['ticket_number']   = [str(ticket)[6:25].strip() for ticket in sirena_filtered['ticket_number'].values]


In [541]:
sirena_filtered.head()

,flight_number,ticket_number,travel_doc,from,dest
id,,,,,
0,SU1306,7360415302044672,9375 053270,Moscow,Novosibirsk
1,SU1323,7398421117936516,2244 645520,Murmansk,Moscow
2,SU1481,5174973140468001,8115 961316,Krasnoyarsk,Moscow
3,SU1180,5274206497242737,98 6865148,Moscow,Volgograd
4,SU1180,6247422701565929,4396 926588,Moscow,Volgograd


### Загрузим таблицу BoardingData и приведем ее в удобный вид

In [543]:
boarding = pd.read_csv('../data/BoardingData.csv', sep=';')
boarding.columns = [
    'passenger_first_name',
    'passenger_second_name',
    'passenger_last_name',
    'passenger_sex',
    'passenger_birth_date',
    'passenger_document',
    'booking_code',
    'ticket_number',
    'baggage',
    'flight_date',
    'flight_time',
    'flight_number',
    'code_share',
    'to'
]
boarding.head()

,passenger_first_name,passenger_second_name,passenger_last_name,passenger_sex,passenger_birth_date,passenger_document,booking_code,ticket_number,baggage,flight_date,flight_time,flight_number,code_share,to
0,SAVELII,VIKTOROVICH,RUSANOV,Male,03/10/1983,2879 096860,FRNINO,6625956945991971,Transit,2017-03-22,06:05,SU1369,Own,Moscow
1,LEV,MARKOVICH,ISAEV,Male,12/13/1975,1788 173211,Not presented,1643715499224676,Registered,2017-03-18,22:10,SU1180,Own,Volgograd
2,NIKOLAI,I.,NIKOLSKII,Male,12/26/1990,4396 926588,VWNYGF,6247422701565929,Transit,2017-03-18,22:10,SU1180,Own,Volgograd
3,ANATOLII,PETROVICH,SHILOV,Male,05/24/1997,2595 919752,WQFFUE,Not presented,Registered,2017-03-18,22:10,SU1180,Own,Volgograd
4,MIROSLAVA,VIACHESLAVOVNA,SEMENOVA,Female,01/31/1976,6775 516990,Not presented,Not presented,Registered,2017-03-12,11:45,SU6284,Own,Moscow


In [544]:
boarding.count()

passenger_first_name     155147
passenger_second_name    155147
passenger_last_name      155147
passenger_sex            155147
passenger_birth_date     155147
passenger_document       155147
booking_code             155147
ticket_number            155147
baggage                  116215
flight_date              155147
flight_time              155147
flight_number            155147
code_share               155147
to                       155147
dtype: int64

### Оставим только нужные колонки

In [546]:
boarding_filtered = boarding[['passenger_document', 'ticket_number', 'flight_date', 'flight_time', 'flight_number', 'to']]

In [547]:
boarding_filtered.head()

,passenger_document,ticket_number,flight_date,flight_time,flight_number,to
0,2879 096860,6625956945991971,2017-03-22,06:05,SU1369,Moscow
1,1788 173211,1643715499224676,2017-03-18,22:10,SU1180,Volgograd
2,4396 926588,6247422701565929,2017-03-18,22:10,SU1180,Volgograd
3,2595 919752,Not presented,2017-03-18,22:10,SU1180,Volgograd
4,6775 516990,Not presented,2017-03-12,11:45,SU6284,Moscow


### Добавим полные имена в таблицу

In [549]:
boarding_filtered['name'] = boarding['passenger_first_name'] + ' ' + boarding['passenger_last_name']

C:\Users\Hp\AppData\Local\Temp\ipykernel_5096\1651273356.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boarding_filtered['name'] = boarding['passenger_first_name'] + ' ' + boarding['passenger_last_name']


In [550]:
boarding_filtered.head()

,passenger_document,ticket_number,flight_date,flight_time,flight_number,to,name
0,2879 096860,6625956945991971,2017-03-22,06:05,SU1369,Moscow,SAVELII RUSANOV
1,1788 173211,1643715499224676,2017-03-18,22:10,SU1180,Volgograd,LEV ISAEV
2,4396 926588,6247422701565929,2017-03-18,22:10,SU1180,Volgograd,NIKOLAI NIKOLSKII
3,2595 919752,Not presented,2017-03-18,22:10,SU1180,Volgograd,ANATOLII SHILOV
4,6775 516990,Not presented,2017-03-12,11:45,SU6284,Moscow,MIROSLAVA SEMENOVA


### Отсутствует информация о месте вылета. Дополним недостающую информацию с таблицы расписания

In [552]:
time_table = pd.read_csv('../data/SkyTeam_Timetable.csv')
time_table.columns = [
    "from",
    "from_code",
    "to",
    "to_code",
    'validity',
    'days',
    "flight_time",
    "arr_time",
    "flight_number",
    "aircraft",
    "travel_time"
]

In [553]:
time_table.head()

,from,from_code,to,to_code,validity,days,flight_time,arr_time,flight_number,aircraft,travel_time
0,"Culiacan , Mexico",CUL,"Mexico City , Mexico",MEX,03 Nov - 03 Nov,7,06:00,08:55,AM169,E90,1H55M
1,"Mexico City , Mexico",MEX,"Culiacan , Mexico",CUL,08 Dec - 08 Dec,7,06:50,07:52,AM162,E90,2H2M
2,"Culiacan , Mexico",CUL,"Mexico City , Mexico",MEX,19 Nov - 24 Nov,234567,06:00,08:55,AM169,E90,1H55M
3,"Mexico City , Mexico",MEX,"Culiacan , Mexico",CUL,29 Nov - 29 Nov,5,06:55,08:01,AM162,E90,2H6M
4,"Culiacan , Mexico",CUL,"Mexico City , Mexico",MEX,07 Jan - 31 Jan,234567,06:01,08:50,AM169,E90,1H49M


### Приведем названия городов в удобный вид

In [555]:
time_table['from'] = [str(city).split(',')[0].strip() for city in time_table['from']]
time_table['to'] = [str(city).split(',')[0].strip() for city in time_table['to']]

In [556]:
time_table.head()

,from,from_code,to,to_code,validity,days,flight_time,arr_time,flight_number,aircraft,travel_time
0,Culiacan,CUL,Mexico City,MEX,03 Nov - 03 Nov,7,06:00,08:55,AM169,E90,1H55M
1,Mexico City,MEX,Culiacan,CUL,08 Dec - 08 Dec,7,06:50,07:52,AM162,E90,2H2M
2,Culiacan,CUL,Mexico City,MEX,19 Nov - 24 Nov,234567,06:00,08:55,AM169,E90,1H55M
3,Mexico City,MEX,Culiacan,CUL,29 Nov - 29 Nov,5,06:55,08:01,AM162,E90,2H6M
4,Culiacan,CUL,Mexico City,MEX,07 Jan - 31 Jan,234567,06:01,08:50,AM169,E90,1H49M


### Оставим только нужные колонки

In [558]:
time_table_filtered = time_table[['from', 'to', 'flight_time', 'flight_number']]

In [559]:
time_table_filtered.head()

,from,to,flight_time,flight_number
0,Culiacan,Mexico City,06:00,AM169
1,Mexico City,Culiacan,06:50,AM162
2,Culiacan,Mexico City,06:00,AM169
3,Mexico City,Culiacan,06:55,AM162
4,Culiacan,Mexico City,06:01,AM169


### Заполним недостающую информациюю в таблице Boarding

In [561]:
boarding_full = pd.merge(boarding_filtered, time_table_filtered, how='left', on=['flight_number', 'flight_time']) 
boarding_full.drop_duplicates(inplace=True)
boarding_full.dropna(inplace=True)
boarding_full.reset_index(inplace=True)

In [562]:
boarding_full.head()

,index,passenger_document,ticket_number,flight_date,flight_time,flight_number,to_x,name,from,to_y
0,0,2879 096860,6625956945991971,2017-03-22,06:05,SU1369,Moscow,SAVELII RUSANOV,Stavropol,Moscow
1,4,1788 173211,1643715499224676,2017-03-18,22:10,SU1180,Volgograd,LEV ISAEV,Moscow,Volgograd
2,12,4396 926588,6247422701565929,2017-03-18,22:10,SU1180,Volgograd,NIKOLAI NIKOLSKII,Moscow,Volgograd
3,20,2595 919752,Not presented,2017-03-18,22:10,SU1180,Volgograd,ANATOLII SHILOV,Moscow,Volgograd
4,28,6775 516990,Not presented,2017-03-12,11:45,SU6284,Moscow,MIROSLAVA SEMENOVA,Yuzhno,Moscow


In [563]:
boarding_full

,index,passenger_document,ticket_number,flight_date,flight_time,flight_number,to_x,name,from,to_y
0,0,2879 096860,6625956945991971,2017-03-22,06:05,SU1369,Moscow,SAVELII RUSANOV,Stavropol,Moscow
1,4,1788 173211,1643715499224676,2017-03-18,22:10,SU1180,Volgograd,LEV ISAEV,Moscow,Volgograd
2,12,4396 926588,6247422701565929,2017-03-18,22:10,SU1180,Volgograd,NIKOLAI NIKOLSKII,Moscow,Volgograd
3,20,2595 919752,Not presented,2017-03-18,22:10,SU1180,Volgograd,ANATOLII SHILOV,Moscow,Volgograd
4,28,6775 516990,Not presented,2017-03-12,11:45,SU6284,Moscow,MIROSLAVA SEMENOVA,Yuzhno,Moscow
...,...,...,...,...,...,...,...,...,...,...
136888,1671877,0282 596160,3990646011873025,2017-05-18,06:30,SU1158,Rostov,IULIIA GORIUNOVA,Moscow,Rostov
136889,1671885,3256 077151,8653871661040452,2017-05-18,06:30,SU1158,Rostov,RAMIL DOROFEEV,Moscow,Rostov
136890,1671893,3497 458395,0223584175158947,2017-05-18,06:30,SU1158,Rostov,RUSLAN CHISTIAKOV,Moscow,Rostov
136891,1671902,4567 242088,0975693658645383,2017-09-16,07:00,SU1640,Omsk,VIACHESLAV KONONOV,Moscow,Omsk


### Объединим таблицы Sirena и Boarding по номеру полета и по номеру билета

In [565]:
merged_df = pd.merge(sirena_filtered, boarding_full, how='inner', on=['ticket_number','flight_number'])

In [566]:
merged_df

,flight_number,ticket_number,travel_doc,from_x,dest,index,passenger_document,flight_date,flight_time,to_x,name,from_y,to_y
0,SU1180,6247422701565929,4396 926588,Moscow,Volgograd,12,4396 926588,2017-03-18,22:10,Volgograd,NIKOLAI NIKOLSKII,Moscow,Volgograd
1,SU6284,7467749130398378,0058 142289,Yuzhno,Moscow,38,0058 142289,2017-03-12,11:45,Moscow,ARTEM KAPUSTIN,Yuzhno,Moscow
2,SU1281,5954073786122008,53 7554162,Kazan,Moscow,48,6600 251370,2017-07-29,14:15,Moscow,ZARINA TITOVA,Kazan,Moscow
3,SU1281,2264717979478322,0078 271703,Kazan,Moscow,58,0078 271703,2017-07-29,14:15,Moscow,SAMIR BARSUKOV,Kazan,Moscow
4,SU1482,1678047813210634,0916 040520,Moscow,Krasnoyarsk,80,0916 040520,2017-04-07,00:15,Krasnoyarsk,ALBINA KONSTANTINOVA,Moscow,Krasnoyarsk
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34244,SU1337,7210398086703616,1173 742866,Arkhangelsk,Moscow,1671677,1173 742866,2017-01-08,21:10,Moscow,ARTEM IAKOVLEV,Arkhangelsk,Moscow
34245,SU1337,9975783952109986,3242 441269,Arkhangelsk,Moscow,1671697,3242 441269,2017-01-08,21:10,Moscow,GEORGII OZEROV,Arkhangelsk,Moscow
34246,SU1181,7140622416563329,57 3978597,Volgograd,Moscow,1671737,0781 620871,2017-02-14,03:10,Moscow,AMINA SUSLOVA,Volgograd,Moscow
34247,SU1181,1569746161308219,7351 760618,Volgograd,Moscow,1671837,7351 760618,2017-02-14,03:10,Moscow,BORIS GOROKHOV,Volgograd,Moscow


### Аномалии в маршрутах рейсов

In [689]:
routes_anomaly = merged_df[merged_df['from_x'] != merged_df['from_y']].reset_index()

In [695]:
routes_anomaly.head()

,level_0,flight_number,ticket_number,travel_doc,from_x,dest,index,passenger_document,flight_date,flight_time,to_x,name,from_y,to_y
0,28,SU1253,2060218382789760,9309 028073,Moscow,Nizhnekamsk,1963,9309 028073,2017-01-18,18:00,Nizhnekamsk,KARINA TRIFONOVA,Nizhnekamsk,Moscow
1,31,SU1255,3817709378907543,3410 455773,Moscow,Nizhnekamsk,2100,3410 455773,2017-04-23,11:10,Nizhnekamsk,NINA BELKINA,Nizhnekamsk,Moscow
2,142,SU1521,2234255181059884,1088 627121,Moscow,Novyj Urengoj,7555,1088 627121,2017-02-13,17:20,Novyj Urengoj,IULIIA DUBOVA,Novyj Urengoj,Moscow
3,143,SU1521,3013862518250115,4591 369913,Moscow,Novyj Urengoj,7635,4591 369913,2017-02-13,17:20,Novyj Urengoj,GORDEI POZDNIAKOV,Novyj Urengoj,Moscow
4,245,SU1255,6095152087387152,8114 646635,Moscow,Nizhnekamsk,12622,8114 646635,2017-02-04,11:10,Nizhnekamsk,ODISSEI SVIRIDOV,Nizhnekamsk,Moscow


## Аномалии в паспортных данных

In [703]:
passport_anomaly = merged_df[merged_df['passenger_document'] != merged_df['travel_doc']].reset_index()

In [705]:
passport_anomaly

,level_0,flight_number,ticket_number,travel_doc,from_x,dest,index,passenger_document,flight_date,flight_time,to_x,name,from_y,to_y
0,2,SU1281,5954073786122008,53 7554162,Kazan,Moscow,48,6600 251370,2017-07-29,14:15,Moscow,ZARINA TITOVA,Kazan,Moscow
1,12,SU1641,7981389337233151,64 6936535,Omsk,Moscow,501,5455 791120,2017-02-14,15:20,Moscow,TAMARA EGOROVA,Omsk,Moscow
2,34,SU6286,2715934247404128,64 0970389,Petropavlovsk,Moscow,2229,7039 733246,2017-01-06,16:30,Moscow,MARK MALAKHOV,Petropavlovsk,Moscow
3,51,SU1443,0750288320194269,62 4199368,Irkutsk,Moscow,3047,0213 604722,2017-03-16,09:55,Moscow,RAMIL LOGINOV,Irkutsk,Moscow
4,53,SU1253,2488789052239469,03 4905603,Nizhnekamsk,Moscow,3325,6096 934457,2017-01-21,17:40,Moscow,RADMIR KLIMOV,Nizhnekamsk,Moscow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3326,34166,SU1198,6980490540175550,44 7356491,Moscow,Kazan,1668035,9398 716767,2017-03-27,22:40,Kazan,IGNAT MURAVEV,Moscow,Kazan
3327,34171,SU1461,2540126553762897,36 2402208,Novosibirsk,Moscow,1668149,9834 775970,2017-01-21,18:30,Moscow,LARISA USPENSKAIA,Novosibirsk,Moscow
3328,34172,SU1139,0407050015055318,24 6268908,Sochi,Moscow,1668158,1351 091752,2017-09-13,09:50,Moscow,KIRILL KRAVTSOV,Sochi,Moscow
3329,34208,SU1135,5452012200471669,87 9527029,Sochi,Moscow,1669922,8071 518212,2017-05-20,14:10,Moscow,VERONIKA TOLKACHEVA,Sochi,Moscow


## Попробуем найти аномалии в расписании рейсов

### Отберем нужные колонки

In [711]:
sub_df = merged_df[['name', 'flight_number', 'ticket_number', 'flight_date']]

In [713]:
sub_df.head()

,name,flight_number,ticket_number,flight_date
0,NIKOLAI NIKOLSKII,SU1180,6247422701565929,2017-03-18
1,ARTEM KAPUSTIN,SU6284,7467749130398378,2017-03-12
2,ZARINA TITOVA,SU1281,5954073786122008,2017-07-29
3,SAMIR BARSUKOV,SU1281,2264717979478322,2017-07-29
4,ALBINA KONSTANTINOVA,SU1482,1678047813210634,2017-04-07


In [716]:
time_table.head()

,from,from_code,to,to_code,validity,days,flight_time,arr_time,flight_number,aircraft,travel_time
0,Culiacan,CUL,Mexico City,MEX,03 Nov - 03 Nov,7,06:00,08:55,AM169,E90,1H55M
1,Mexico City,MEX,Culiacan,CUL,08 Dec - 08 Dec,7,06:50,07:52,AM162,E90,2H2M
2,Culiacan,CUL,Mexico City,MEX,19 Nov - 24 Nov,234567,06:00,08:55,AM169,E90,1H55M
3,Mexico City,MEX,Culiacan,CUL,29 Nov - 29 Nov,5,06:55,08:01,AM162,E90,2H6M
4,Culiacan,CUL,Mexico City,MEX,07 Jan - 31 Jan,234567,06:01,08:50,AM169,E90,1H49M


### Обьединим поддатасет с таблицей расписаний

In [839]:
df = pd.merge(sub_df, time_table[['flight_number', 'validity']], how='inner', on='flight_number').drop_duplicates()

In [843]:
df.head()

,name,flight_number,ticket_number,flight_date,validity
0,NIKOLAI NIKOLSKII,SU1180,6247422701565929,2017-03-18,01 Nov - 30 Dec
1,NIKOLAI NIKOLSKII,SU1180,6247422701565929,2017-03-18,01 Jan - 31 Jan
20,ARTEM KAPUSTIN,SU6284,7467749130398378,2017-03-12,01 Nov - 31 Jan
30,ZARINA TITOVA,SU1281,5954073786122008,2017-07-29,01 Nov - 31 Jan
40,SAMIR BARSUKOV,SU1281,2264717979478322,2017-07-29,01 Nov - 31 Jan


### Приведем колонки дат к удобному виду

In [845]:
df['start_date'] = [pd.to_datetime(date.split(' - ')[0], format='%d %b').strftime('%m-%d') for date in df['validity'].values]
df['end_date']  = [pd.to_datetime(date.split(' - ')[1], format='%d %b').strftime('%m-%d') for date in df['validity'].values]
df['flight_date'] = [pd.to_datetime(date, format='%Y-%m-%d').strftime('%m-%d') for date in df['flight_date'].values]

In [847]:
df.head()

,name,flight_number,ticket_number,flight_date,validity,start_date,end_date
0,NIKOLAI NIKOLSKII,SU1180,6247422701565929,03-18,01 Nov - 30 Dec,11-01,12-30
1,NIKOLAI NIKOLSKII,SU1180,6247422701565929,03-18,01 Jan - 31 Jan,01-01,01-31
20,ARTEM KAPUSTIN,SU6284,7467749130398378,03-12,01 Nov - 31 Jan,11-01,01-31
30,ZARINA TITOVA,SU1281,5954073786122008,07-29,01 Nov - 31 Jan,11-01,01-31
40,SAMIR BARSUKOV,SU1281,2264717979478322,07-29,01 Nov - 31 Jan,11-01,01-31


### Проверим условия вхождения даты полета в интервал расписания

In [849]:
df['correct_date'] = (df['start_date'] <= df['flight_date']) & (df['flight_date'] <= df['end_date'])

In [855]:
schedule_anomaly = df[df['correct_date'] == False].drop_duplicates().reset_index()

## Объединим все списки аномальных пассажиров

In [859]:
df = pd.merge(routes_anomaly[['name']], passport_anomaly[['name']], how='outer', on='name') 

In [863]:
anomaly_passengers = pd.merge(df[['name']], schedule_anomaly[['name']], how='outer', on='name')

In [ ]:
anomaly_passengers.drop_duplicates(inplace=True)